# Import Packages

In [20]:
# built-in
from os import path

# third-party (pip install required)
import numpy as np
import pandas as pd
from pymodulon.compare import compare_ica
from pymodulon.core import IcaData
from pymodulon.io import load_json_model, save_to_json
from pymodulon.util import explained_variance
from tqdm.notebook import tqdm

In [2]:
# Enter the location of your data here
data_dir = '../../data/pure_precise/'
annotation_dir = '../../data/annotation'

# QC Tables

## Metadata

Your metadata file will probably have a lot of columns, most of which you may not care about. Feel free to save a secondary copy of your metadata file with only columns that seem relevant to you. The two most important columns are:
1. `project`
2. `condition`

Make sure that these columns exist in your metadata file

In [3]:
df_metadata = pd.read_csv(path.join(data_dir,'metadata.csv'),index_col=0)
df_metadata[['project','condition']].head()

,project,condition
ecoli_00001,control,wt_glc
ecoli_00002,control,wt_glc
ecoli_00003,fur,wt_dpd
ecoli_00004,fur,wt_dpd
ecoli_00005,fur,wt_fe


In [4]:
print(df_metadata.project.notnull().all())
print(df_metadata.condition.notnull().all())

True
True


## TRN

Each row of the TRN file represents a regulatory interaction.  
**Your TRN file must have the following columns:**
1. `regulator` - Name of regulator (`/` or `+` characters will be converted to `;`)
1. `gene_id` - Locus tag of gene being regulated

The following columns are optional, but are helpful to have:
1. `regulator_id` - Locus tag of regulator
1. `gene_name` - Name of gene (can automatically update this using `name2num`)
1. `direction` - Direction of regulation ('+' for activation, '-' for repression, '?' or NaN for unknown)
1. `evidence` - Evidence of regulation (e.g. ChIP-exo, qRT-PCR, SELEX, Motif search)
1. `PMID` - Reference for regulation

You may add any other columns that could help you. TRNs may be saved as either CSV or TSV files. See below for an example:

In [5]:
df_trn = pd.read_csv(path.join(annotation_dir,'TRN.csv'))
df_trn.head()

,regulator,gene_name,gene_id,effect,evidence,source
0,AccB,accB,b3255,-,0.0,RegulonDB_TF
1,RpoD,pheU,b4134,+,0.0,RegulonDB_sigma
2,RpoD,pheP,b0576,+,0.0,RegulonDB_sigma
3,RpoD,pheL,b2598,+,0.0,RegulonDB_sigma
4,RpoD,pheA,b2599,+,0.0,RegulonDB_sigma


The `regulator` and `gene_id` must be filled in for each row

In [6]:
print(df_trn.regulator.notnull().all())
print(df_trn.gene_id.notnull().all())

True
True


# Create IcaData

In [13]:
ica_data = IcaData(
    M=path.join(data_dir,'M.csv'),
    A=path.join(data_dir,'A.csv'),
    X=path.join(data_dir, 'log_tpm_norm.csv'),
    gene_table=path.join(annotation_dir,'gene_info.csv'),
    sample_table=path.join(data_dir,'metadata.csv'),
    trn=path.join(annotation_dir,'TRN.csv'),
    optimize_cutoff=True
)

# TRN Enrichment
Use `compute_trn_enrichment` to automatically check for Regulatory iModulons. The more complete your TRN, the more regulatory iModulons you'll find.

For this automated step, use strict cutoff for evidence required (can loosen later during manual annotation)

In [24]:
ica_data.compute_trn_enrichment(max_regs=2, evidence=[1, 2], save=True)

/Users/cam/.virtualenvs/ica/lib/python3.10/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/Users/cam/.virtualenvs/ica/lib/python3.10/site-packages/pymodulon/core.py:610: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  keep_cols = self.imodulon_table.loc[


,imodulon,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs
0,1,NrdR/Fur,1.193362e-64,4.287035e-60,0.688525,0.552632,0.613139,42.0,76.0,61.0,2.0
1,1,NfeR/Fur,6.501981e-60,1.167886e-55,0.639344,0.565217,0.600000,39.0,69.0,61.0,2.0
2,1,Fur,2.937251e-58,3.517260e-54,0.622951,0.567164,0.593750,38.0,67.0,61.0,1.0
3,1,Fur/GadE,7.411942e-56,5.325332e-52,0.639344,0.475610,0.545455,39.0,82.0,61.0,2.0
4,1,RutR/Fur,7.411942e-56,5.325332e-52,0.639344,0.475610,0.545455,39.0,82.0,61.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
1019,105,Lrp/Fnr,1.283712e-08,6.406437e-06,0.260000,0.084967,0.128079,13.0,153.0,50.0,2.0
1020,105,Fnr,1.498567e-08,7.376236e-06,0.220000,0.108911,0.145695,11.0,101.0,50.0,1.0
1021,105,AdiY/OmpR,1.568017e-08,7.613784e-06,0.160000,0.195122,0.175824,8.0,41.0,50.0,2.0
1022,105,IHF,2.053558e-08,9.838459e-06,0.260000,0.081761,0.124402,13.0,159.0,50.0,1.0


# Single-Gene-Dominant iModulons

In [8]:
sg_imods = ica_data.find_single_gene_imodulons(save=True)

In [9]:
for sg_mod in sg_imods:
    sg_name = ica_data.view_imodulon(sg_mod).sort_values(
        by='gene_weight', ascending=False).iloc[0, :]['gene_name']
    ica_data.rename_imodulons({sg_mod:f'{sg_name}'})

# iModulon Names

Convenient to string-ify all names so as not to have a mix of strings and ints in the index

In [10]:
rename_dict = {}
for im_name in ica_data.imodulon_names:
    if isinstance(im_name, int):
        rename_dict[im_name] = str(im_name)
ica_data.rename_imodulons(rename_dict)

# Explained Variance

In [14]:
exp_vars = [explained_variance(ica_data, imodulons=[imod]) for imod in ica_data.imodulon_names]

In [15]:
ica_data.imodulon_table['exp_var'] = exp_vars

# iModulon Size

In [16]:
ica_data.imodulon_table['imodulon_size'] = [ica_data.view_imodulon(imod).shape[0]
                                            for imod in ica_data.imodulon_names]

# PRECISE-1K Correlations

## Load PRECISE-1K

In [17]:
precise1k = load_json_model('../../data/precise1k/precise1k.json.gz')

## Run Correlation Comparisons

In [18]:
match_rows = []
for match in compare_ica(precise1k.M, ica_data.M, method='pearson')[0]:
    match_rows.append({
        'dataset': 'PRECISE-1K',
        'p1K_iM': match[0],
        'pure_im': match[1],
        'method': 'pearson',
        'corr': match[2]
    })
for match in compare_ica(precise1k.M, ica_data.M, method='spearman')[0]:
    match_rows.append({
        'dataset': 'PRECISE-1K',
        'p1K_iM': match[0],
        'pure_im': match[1],
        'method': 'spearman',
        'corr': match[2]
    })
    
match_df = pd.DataFrame(match_rows)

In [21]:
for im, im_row in ica_data.imodulon_table.iterrows():
    im_match = match_df[match_df['pure_im'] == im]
    if not im_match.empty:
        max_corr = np.argmax(im_match['corr'])
        max_corr_im = im_match.iloc[max_corr]['p1K_iM']
        max_corr_im_df = im_match[im_match['p1K_iM'] == max_corr_im]
        for _, max_corr_row in max_corr_im_df.iterrows():
            ica_data.imodulon_table.loc[im, 'PRECISE-1K'] = max_corr_row['p1K_iM']
            ica_data.imodulon_table.loc[im, max_corr_row['method']] = max_corr_row['corr']

# Re-Save M/A Matrices and iModulon Table

In [27]:
ica_data.imodulon_table.to_csv(path.join(data_dir, 'imodulon_table.csv'))
ica_data.A.to_csv(path.join(data_dir, 'A.csv'))
ica_data.M.to_csv(path.join(data_dir, 'M.csv'))

# Save IcaData object

This will save your iModulon table, your thresholds, and any other information stored in the ica_data object.

In [28]:
save_to_json(ica_data, path.join(data_dir, 'pure_precise'), compress=True)